<h1>Preparing Corpus for STM - IDeaS Standard Notebook for STM Rendering</h1>
<p>The aim of this notebook is process the csv file, prep for use with STM in R</p>

In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import *

In [ ]:
import pandas as pd
import numpy as np
import xml

In [ ]:
from tqdm import tqdm # progress bar
from nltk.util import ngrams
import collections
from collections import Counter

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#import spacy
import re
import xlsxwriter

In [ ]:
# excel files tend to presure line spaces well
df = pd.read_excel("corpus_raw.xlsx")

In [ ]:
df.info()

In [ ]:
df.head(2)

#### What variables are we going to use here as covariates? 

In [ ]:
df['source'].value_counts()

Let's use date and source. However, in order to do this, we will need to process each. For date, we will need a relative date measure (# weeks, # months). For source, we will need to consolidate and code for political leaning.

##### need to code for relative date (counting up from Jan 18, 2020)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
def count_days_from_origin(row):
    date_origin = pd.to_datetime('2020-01-18')
    return (row['date']- date_origin).days + 1 # with R, we want to start with 1

In [ ]:
df['days_since_origin'] = df.apply(count_days_from_origin, axis=1)

In [ ]:
def count_months_from_origin(row):
    date_origin = pd.to_datetime('2020-01-18')
    r = relativedelta(row['date'], date_origin)
    months_difference = (r.years * 12) + r.months + 1 # with R, we want to start with 1
    return months_difference

In [ ]:
df['months_since_origin'] = df.apply(count_months_from_origin, axis=1)

In [ ]:
df.head()

In [ ]:
df.info()

Not all articles have author (byline), so removing it for consistency

In [ ]:
df.drop('author', axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
df['days_since_origin'].max()

In [ ]:
df['days_since_origin'].mean()

In [ ]:
df['days_since_origin'].min()

In [ ]:
df['months_since_origin'].max()

In [ ]:
df['months_since_origin'].mean()

In [ ]:
df['months_since_origin'].min()

#### Additional pre-processing

In [ ]:
df_for_stm = df.copy()

In [ ]:
# simple approach for stripping HTML code
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)


# this method doesn't work
#def remove_tags(text):
#    ''.join(xml.etree.ElementTree.fromstring(text).itertext())

In [ ]:
def clean_string(text):
    text = text.lower()
    text = text.replace(",", "")
    text = text.replace('"', '')
    text = text.replace("’", "'") # straighten quotes
    text = text.replace("'s", "")
    text = text.replace("'ve", "")
    text = text.replace("'re", "")
    text = text.replace("Canada'", "Canada")
    text = text.replace(":", "")
    text = text.replace("%", "")
    text = text.replace("(", "")
    text = text.replace(")", "")
    text = text.replace("*", "")        
    text = text.replace("?", "")
    text = text.replace("“", "")
    text = text.replace(";", "")    
    text = text.replace("</a", "")
    text = re.sub(r'http\S+', '', text)    
    text = re.sub(r'[\w\.-]+@[\w\.-]+', '', text) # remove email addresses    
    text = re.sub(r'\b\w+\-*[\.]*\w+\.[com|org|co\.uk]/*[^ ]+\b', '', text) # remove urls        
    text = re.sub("\.+", "", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\-+", " ", text)    
    text = text.strip()
    return text

In [ ]:
# strip HTML
df_for_stm['body'] = df_for_stm['body'].apply(remove_tags)
df_for_stm['body'] =  df_for_stm['body'].apply(clean_string)

## Everything below is custom code written for a project, shown here for illustrative purposes only

### Basic cleaning of Sources

In [ ]:
df_for_stm['source'].value_counts()

In [ ]:
df_for_stm['source'].nunique()

In [ ]:
def clean_sources(text):
    text = text.replace("(Breaking News)", " ")
    text = text.strip()
    return text

In [ ]:
df_for_stm['source']= df_for_stm['source'].apply(clean_sources)

In [ ]:
df_for_stm['source'].value_counts()

In [ ]:
df_for_stm['source'].value_counts().to_excel("df_for_stm_source_value_counts.xlsx")

### Qualitatively Coding sources for political leanings

We discussed coding newspapers for political leanings. For sources we were not able to clearly code, we are dropping them from the analysis.

In [ ]:
sources = ['Vancouver Province',  'Winnipeg Free Press',
'Montreal Gazette', 'The Globe and Mail', 'The Toronto Star', 'The Globe and Mail',
'National Post', 'Calgary Herald']

In [ ]:
df_for_stm = df_for_stm[df_for_stm['source'].isin(sources)].copy()

In [ ]:
df_for_stm.info()

This move trimmed our dataframe to 3789 rows

<h4>Code for political leaning</h4>

In [ ]:
sources = df_for_stm['source'].tolist()

In [ ]:
political_leaning = []
for s in sources:
    if s=='Vancouver Province': political_leaning.append("C")
    elif s=='Winnipeg Free Press': political_leaning.append("C")
    elif s=='Montreal Gazette': political_leaning.append("L")
    elif s=='The Toronto Star': political_leaning.append("L")
    elif s=='The Globe and Mail': political_leaning.append("L")
    elif s=='National Post': political_leaning.append("C")
    elif s=='Calgary Herald': political_leaning.append("C")    

In [ ]:
df_for_stm['political_leaning']=political_leaning

In [ ]:
df_for_stm.head(2)

### Save for further analysis in R

In [ ]:
filename = "covid_mask_canada_for_STM"
df_for_stm.to_csv(filename+".csv") # better for STM analysis
df_for_stm.to_excel(filename+".xlsx") # better for quick viewing

Go to R directly for more processing.. note, this assumes there is an adjacent folder named "rendering_topics"

In [ ]:
import shutil
shutil.copy(filename+".csv", '../rendering_topics/')